In [ ]:
!pip install catboost

In [ ]:
#импорт библиотек
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from catboost import CatBoostClassifier, Pool, cv, MetricVisualizer
from sklearn.metrics import classification_report
import re
import itertools

In [ ]:
#подключение google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# path = '/Users/mikhailgoncharov/Downloads/'

In [ ]:
# data = pd.read_csv(path + 'skills_for_prediction_model.csv', usecols = ['id','name','skills'])
data = pd.read_csv('/content/drive/MyDrive/ML model/skills_for_prediction_model.csv', usecols = ['id','name','skills'])

In [ ]:
#приводим датасет к необходимому виду
df = pd.get_dummies(data, columns=['skills']).copy()
df.columns = [col[7:] if 'skills_' in col else col for col in df.columns ]

In [ ]:
df2 = df.copy()

In [ ]:
df.shape,df2.shape

((5357, 720), (5357, 720))

In [ ]:
#очищаем текст от символов
def clear_text(text):
    
    text_cleared = re.sub(r"[^a-zA-Zа-яА-ЯёЁ\' ]", ' ', text) #очистка текста
    
    return " ".join(text_cleared.split())  
    

In [ ]:
#приводим наименования позиций к общим названиям
def replace_name(raw_series, target_name, exceptions=[]):
  a = set(target_name.split())
  return pd.Series([target_name if val not in exceptions and a == a.intersection(set(clear_text(val).lower().split())) else val for val in raw_series])
  

In [ ]:
#составляем лист 
levels = ['junior', 'middle', 'senior']
entire_positions = ['data analyst', 'data scientist', 'data engineer', 'ml engineer']
positions = [ a + " " + b for a,b in list(itertools.product(levels,entire_positions))]
[positions.append(pos) for pos in entire_positions]
positions

['junior data analyst',
 'junior data scientist',
 'junior data engineer',
 'junior ml engineer',
 'middle data analyst',
 'middle data scientist',
 'middle data engineer',
 'middle ml engineer',
 'senior data analyst',
 'senior data scientist',
 'senior data engineer',
 'senior ml engineer',
 'data analyst',
 'data scientist',
 'data engineer',
 'ml engineer']

In [ ]:
#заменяем наименования позиций
replaced = df['name']
for pos in positions:
  print(pos, 'has been replaced')
  replaced = replace_name(replaced, pos, positions[:-4])

junior data analyst has been replaced
junior data scientist has been replaced
junior data engineer has been replaced
junior ml engineer has been replaced
middle data analyst has been replaced
middle data scientist has been replaced
middle data engineer has been replaced
middle ml engineer has been replaced
senior data analyst has been replaced
senior data scientist has been replaced
senior data engineer has been replaced
senior ml engineer has been replaced
data analyst has been replaced
data scientist has been replaced
data engineer has been replaced
ml engineer has been replaced


In [ ]:
#так же готовим датасет с более усреднёнными позициями (без разбивки на джуна, мида и сеньора)
replaced2 = df['name']
for pos in positions:
  print(pos, 'has been replaced')
  replaced2 = replace_name(replaced, pos)

junior data analyst has been replaced
junior data scientist has been replaced
junior data engineer has been replaced
junior ml engineer has been replaced
middle data analyst has been replaced
middle data scientist has been replaced
middle data engineer has been replaced
middle ml engineer has been replaced
senior data analyst has been replaced
senior data scientist has been replaced
senior data engineer has been replaced
senior ml engineer has been replaced
data analyst has been replaced
data scientist has been replaced
data engineer has been replaced
ml engineer has been replaced


In [ ]:
df['name'] = replaced

In [ ]:
df2['name'] = replaced2

In [ ]:
#удаляем дубликаты
df.drop_duplicates(inplace = True)

In [ ]:
df2.drop_duplicates(inplace = True)

In [ ]:
#оставляем только вакансии для нашего набора позиций
df_fine = pd.DataFrame(columns=df.columns)
for pos in positions:
  df_fine = df_fine.append(df[df['name'] == pos])


In [ ]:
df_fine2 = pd.DataFrame(columns=df.columns)
for pos in entire_positions:
  df_fine2 = df_fine2.append(df[df['name'] == pos])

In [ ]:
df_fine.shape,df_fine2.shape

((3779, 720), (3085, 720))

In [ ]:
#определяем скилы, не попавшие ни в одну вакансию
zero_cols = []
for col in df_fine.columns:
  if len(df_fine[col].value_counts()/df_fine.shape[0]) == 1:
    zero_cols.append(col)

In [ ]:
zero_cols2 = []
for col in df_fine2.columns:
  if len(df_fine2[col].value_counts()/df_fine2.shape[0]) == 1:
    zero_cols2.append(col)

In [ ]:
#удаляем их

df_fine.drop(zero_cols, axis=1, inplace=True)

In [ ]:
df_fine2.drop(zero_cols2, axis=1, inplace=True)

In [ ]:
df_fine2.shape, df_fine.shape, data.shape


((3085, 457), (3779, 527), (5357, 3))

In [ ]:
# Модели

In [ ]:
target = 'name'

In [ ]:
X, y = df_fine.drop(target, axis=1), df_fine[target]
X.drop(['id'], axis=1,inplace=True)


In [ ]:
X2, y2 = df_fine2.drop(target, axis=1), df_fine2[target]
X2.drop(['id'], axis=1,inplace=True)

In [ ]:
# evaluate a logistic regression model using k-fold cross-validation
from sklearn.model_selection import cross_val_score

# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = LogisticRegression()
# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Accuracy: 0.532 (0.028)


In [ ]:
# evaluate model
scores2 = cross_val_score(model, X2, y2, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(scores2), np.std(scores2)))

Accuracy: 0.654 (0.020)


In [ ]:
# kf = KFold(n_splits=10)
# kf.get_n_splits(X)

# means, mins, maxs = list(),list(),list()

# for train_index, test_index in kf.split(X):
#     #  print("TRAIN:", train_index, "TEST:", test_index)
#      y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    

#      logit  = LogisticRegression()
#      logit.fit(X_train, y_train)
#      scores = cross_val_score(logit, X_train, y_train, scoring='accuracy')
#      means.append(np.mean(scores))
#      mins.append(min(scores))
#      maxs.append(max(scores))
#     #  y_pred = logit.predict(X_test)

In [ ]:
# catboost

In [ ]:
X_train_boost, X_test_boost, y_train_boost, y_test_boost = train_test_split(X, y, test_size=0.2, random_state=1)
X_train_boost, X_val_boost, y_train_boost, y_val_boost = train_test_split(X_train_boost, y_train_boost, test_size=0.25, random_state=1) 
print(f'Train size: {round(X_train_boost.shape[0] / X.shape[0]*100 ,2)}%\
      \nVal size:   {round(X_val_boost.shape[0] / X.shape[0]*100 ,2)}%\
      \nTest size:  {round(X_test_boost.shape[0] / X.shape[0]*100, 2)}%')


Train size: 59.99%      
Val size:   20.01%      
Test size:  20.01%


In [ ]:
train_pool = Pool(
    data=X_train_boost, 
    label=y_train_boost
)

val_pool = Pool(
    data=X_val_boost, 
    label=y_val_boost
)

test_pool = Pool(
    data=X_test_boost, 
    label=y_test_boost
)

In [ ]:
model = CatBoostClassifier(
    iterations=3000,
    early_stopping_rounds=1000,
    learning_rate=0.3,
    eval_metric='Accuracy'
    # snapshot_file='snapshot.bkp',
    # snapshot_interval=60
    # train_dir='raw_num_data'
)

model.fit(
    train_pool,
    eval_set=val_pool,
    verbose=False,
    plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
y_pred = model.predict(X_test_boost)

In [ ]:
print(classification_report(y_test_boost, y_pred))
# print('Train score: {:.3f}'.format(model.score(X_train, y_train)))
# print('Test score: {:.3f}'.format(model.score(X_test, y_test)))

                       precision    recall  f1-score   support

         data analyst       0.60      0.58      0.59       278
        data engineer       0.33      0.04      0.07        26
       data scientist       0.51      0.82      0.63       300
  junior data analyst       0.00      0.00      0.00        16
junior data scientist       0.00      0.00      0.00        16
  middle data analyst       0.00      0.00      0.00        12
 middle data engineer       0.00      0.00      0.00         1
middle data scientist       0.00      0.00      0.00        29
          ml engineer       0.00      0.00      0.00         6
  senior data analyst       0.00      0.00      0.00        32
 senior data engineer       0.00      0.00      0.00         2
senior data scientist       0.00      0.00      0.00        38

             accuracy                           0.54       756
            macro avg       0.12      0.12      0.11       756
         weighted avg       0.44      0.54      0.47 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
X_train_boost, X_test_boost, y_train_boost, y_test_boost = train_test_split(X2, y2, test_size=0.2, random_state=1)
X_train_boost, X_val_boost, y_train_boost, y_val_boost = train_test_split(X_train_boost, y_train_boost, test_size=0.25, random_state=1) 
print(f'Train size: {round(X_train_boost.shape[0] / X.shape[0]*100 ,2)}%\
      \nVal size:   {round(X_val_boost.shape[0] / X.shape[0]*100 ,2)}%\
      \nTest size:  {round(X_test_boost.shape[0] / X.shape[0]*100, 2)}%')


Train size: 48.98%      
Val size:   16.33%      
Test size:  16.33%


In [ ]:
train_pool = Pool(
    data=X_train_boost, 
    label=y_train_boost
)

val_pool = Pool(
    data=X_val_boost, 
    label=y_val_boost
)

test_pool = Pool(
    data=X_test_boost, 
    label=y_test_boost
)

In [ ]:
model = CatBoostClassifier(
    iterations=3000,
    early_stopping_rounds=1000,
    learning_rate=0.3,
    eval_metric='Accuracy',
    # snapshot_file='snapshot.bkp',
    # snapshot_interval=60
    train_dir='/content/drive/MyDrive/ML model'
)

model.fit(
    train_pool,
    eval_set=val_pool,
    verbose=False,
    plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
y_pred2 = model.predict(X_test_boost)

In [ ]:
print(classification_report(y_test_boost, y_pred2))
# print('Train score: {:.3f}'.format(model.score(X_train, y_train)))
# print('Test score: {:.3f}'.format(model.score(X_test, y_test)))

                precision    recall  f1-score   support

  data analyst       0.68      0.54      0.60       278
 data engineer       0.33      0.04      0.07        25
data scientist       0.62      0.80      0.70       308
   ml engineer       0.00      0.00      0.00         6

      accuracy                           0.64       617
     macro avg       0.41      0.34      0.34       617
  weighted avg       0.63      0.64      0.62       617



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#смотрим, какие предикторы оказались самыми важными
np.array(model.get_feature_importance(
    data=train_pool, 
    type='LossFunctionChange', 
    prettified=True
))

array([['Spark', 0.00843773406344539],
       ['Tableau', 0.008339542704222436],
       ['Data Analysis', 0.0066091534124335105],
       ['ETL', 0.006571668750983806],
       ['Математическое моделирование', 0.005112705570169962],
       ['Scrum', 0.0047559527957464764],
       ['Git', 0.004174439130087082],
       ['Big Data', 0.004114546670682118],
       ['Business Intelligence Systems', 0.003810723578818398],
       ['MS SQL', 0.003579931927727764],
       ['machine learning', 0.003553941520038739],
       ['SQL', 0.003515224446568599],
       ['Data Mining', 0.0029970118852938876],
       ['Statistica', 0.0029504093548692323],
       ['ML', 0.0028599697995863593],
       ['Google Analytics', 0.00276910914736872],
       ['Kafka', 0.0027673301587920474],
       ['NLP', 0.0027219231464673532],
       ['Docker', 0.002697502436787458],
       ['Английский язык', 0.002535794770446853],
       ['Analytical skills', 0.00228562263309251],
       ['PyTorch', 0.0022764761265036126],
       

In [ ]:
w = MetricVisualizer('/ML model/')
w.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))